# Jenkins

- Jenkinsfile is a __Pipeline as Code__. It is part of IaC concept.
- Supports both `scripted` (groovy syntax) as well as well as `declarative`(template based) pipeline.
- `Scripted` pipeline starts with __node__ and `declarative` pipeline starts with __pipeline__.

- __when__: For some stage e.g 'test', we only want to run on dev branch or any other branch. For that there is `when` expression.

- environment variable is an importance concept in Jenkins. We can either use the already available tons of Jenkins env var e.g BRANCH_NAME or we can create our own environment variable with `environment {}` expression

- credentials can be used with __1.credentials(for creating credentials in GUI) and 2.credentials binding (for using those credentials in jenkinsfile)__ plugins and then use its function `credentials(id)` to get username and password. This function can be used in our custom environment expression if we need the credentials for multiple stages. If we only need it for one stage, we can use a wrapper `withCredentials()`.

- __tools__: If we want to use some build tools like maven, gradle, jdk etc. for multiple stages, then we can use `tools {}` syntax but the tool name has to be the same as is install in Jenkins (global tool configuration).

- __parameters__: types- string, choice(list), boolean. It's like you define a variable in other programming languages. We can use parameters in `when` expression or any stage as variable.

- `script` block: so that we can modulerize/ make composable jenkinsfile.

- Jenkins need to be installed only on master server and not on agents.

> __jenkins as docker:__ `docker run -p 8080:8080 -p 50000:50000 -v ~/jenkins_home:/var/jenkins_home jenkins/jenkins:lts`

- `# of executors`: number of parallel jobs that it can run.

- __NODE__: Adding a new node means adding a new machine

- Another term for jenkins job is project which is slowly replacing the 'jobs' term. Job or project is a set of automation tasks; e.g "git clone", "run command x", run command y"

__Master__: Jenkins master hosts the user interface and orchestrates jobs between agents and executors.

__Executor__: Slot for running a job. # of executors implies number of parallel jobs that can be run. It's a good idea to use as many executors as the number of CPU cores.


![](./images/jenkins_architecture.png)

__JOB TYPES__:

- The __Pipeline job__ type allows you to define your build pipeline as code in a Jenkinsfile.

- While it could work, a __Freestyle job__ does not allow building pipelines as code.

- The __Multi-configuration job__ type allows you to run freestyle-like jobs with multiple configurations. It's not the optimal solution for Docker pipelines.

# Jenkins Contd.

```groovy
node {
    checkout scm
    /* ... snip ... */
}
```
The `checkout` step will checkout code from source control; `scm` is a special variable which instructs the checkout step to clone the specific revision which triggered this Pipeline run.

### Environment Variable

- An environment directive used in the top-level pipeline block will apply to all steps within the Pipeline.
- An environment directive defined within a stage will only apply the given environment variables to steps within the stage.

### String Interpolation
```groovy
def username = 'Jenkins'
echo 'Hello Mr. ${username}'
echo "I said, Hello Mr. ${username}"
```
Would result in:
```groovy
Hello Mr. ${username}
I said, Hello Mr. Jenkins
```

### Interpolation of sensitive environment variables

> Groovy string interpolation should never be used with credentials.

Groovy string interpolation can leak sensitive environment variables (see: Handling credentials). This is because the sensitive environment variable will be interpolated during Groovy evaluation, and the environment variable’s value could be made available earlier than intended, resulting in sensitive data leaking in various contexts.

```groovy
Jenkinsfile (Declarative Pipeline)
pipeline {
    agent any
    environment {
        EXAMPLE_CREDS = credentials('example-credentials-id')
    }
    stages {
        stage('Example') {
            steps {
                /* WRONG! */
                sh("curl -u ${EXAMPLE_CREDS_USR}:${EXAMPLE_CREDS_PSW} https://example.com/")
                /* CORRECT */
                sh('curl -u $EXAMPLE_CREDS_USR:$EXAMPLE_CREDS_PSW https://example.com/')
                
            }
        }
    }
}
```

Similary, passing special characters `(e.g. / \ $ & % ^ > < | ;)` from a user-defined parameters to shell can result in problems analogous to SQL injection.
```groovy
Jenkinsfile (Declarative Pipeline)
pipeline {
    agent any
    parameters {
        string(name: 'STATEMENT', defaultValue: 'hello; ls /', description: 'What should I say?')
    }
    stages {
        stage('Example') {
            steps {
                /* WRONG! */
                sh("echo ${params.STATEMENT}")
                /* CORRECT */
                sh('echo ${params.STATEMENT}')
            }    
        }
    }
}
```

### Post conditions to handle failure

> `always, unstable, success, failure, changed`

```groovy
Jenkinsfile (Declarative Pipeline)
pipeline {
    agent any
    stages {
        stage('Test') {
            steps {
                sh 'make check'
            }
        }
    }
    post {
        always {
            junit '**/target/*.xml'
        }
        failure {
            mail to: team@example.com, subject: 'The Pipeline failed :('
        }
    }
}
```

---

If you run three shell steps back-to-back, each of those steps has to be started and stopped, requiring connections and resources on the agent and controller to be created and cleaned up. However, if you put all of the commands into a single shell step, then only a single step needs to be started and stopped.

> Instead of creating a series of echo or sh steps, combine them into a single step or script.


# Jenkins Contd. #2

Example of scripted jenkins pipeline with explanation

```groovy
node {
        def app
    
    stage('Clone repository') {
            checkout scm  /* 1 */
    }
    
    stage('Build image') {
            app = docker.build('upengareri/example-image')  /* 2 */
    }
    
    stage('Test') {
        app.inside {  /* 3 */
                sh 'npm test'
        }
    }
    
    stage('Push image') {
        docker.withRegistry('https://registry.hub.docker.com', 'docker-credentials-id') {  /* 4 */
                app.push('latest')  /* 5 */
        }
    }
}
```

1. `scm` is a special variable which instructs the checkout step to clone the specific revision which triggered this Pipeline run.
2. builds the image with the argument inside the build as the name of the image
3. `inside` special syntax that allows running command within a container spawned from the image built above
4. `withRegistry` needs registry url and id for credentials stored by jenkins credentials plugin to access the account
5. push to registry with optional tag

# Best Practices

## Ways of Securing Jenkins

### 1. By running Jenkins in a private network with VPN

![](./images/jenkins_vpn.png)

- In AWS, it can be put in private subnet and used via VPN
- In this case, webhook won't work.

### 2. Run Jenkins with firewall

![](./images/jenkins_firewall.png)

- In AWS, this can be done with _security groups_

### 3. Reverse Proxy via http basic authentication

![](./images/jenkins_reverse_proxy.png)

- Requires two times authentication which might annoy the users


## Managing build dependencies 

- Use docker to wrap the build env as image
- Put the docker image in ECR or other registry so that we don't need to built the environment everytime and using caching (docker layer caching)

# Deployment

- Before deployment we have `BUILD` stage/job that generates the artifacts/executable file let's say jar file which we can deploy to test environment, to artifactory.

- Deployement is finally also done on production environment which is also called `RELEASE`

# Best Practices

- If you have docker organisation, then it is better to create a docker user that has read and write access to the docker repositories and use that user in Jenkins (credential manager)

- We can have a nightly freestyle project that runs only one command `docker system prune -af`. This will clean all the docker images, containers, volumes and build caches.

---

THING TO KEEP IN MIND:

- If you end up doing something inside `docker.build().inside {}` block that results in creating files into workspace that are not owned by Jenkins users, then Jenkins will have problem in cleaning up the workspace. This will result in either Jenkins job failing or will result in running out of disk space by leaving unclean workspace directory after every job invocation. __[Need more clarity here]__

---

USING ECR AS DOCKER REGISTRY:

![](./images/jenkins_ecr.png)

---

> EXAMPLE OF USING SECRET TEXT via credentials manager plugin is when notifying about build failures to slack. Slack incoming webhook generates a secret key, which can be used under secret text dropdown menu of the credentials plugin. This can be used with `withCredentials` block in jenkins scripted pipeline.